In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('mmc4.xlsx')

In [3]:
import pyensembl 
genome = pyensembl.ensembl_grch38

In [4]:
seqs = genome.protein_sequences.fasta_dictionary

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


In [5]:
pep_lengths = set(df.PEP_LEN)

In [6]:
pep_lengths

{8, 9, 10, 11, 12, 13, 14}

In [7]:
from collections import defaultdict
from tqdm import tqdm
kmers_by_size = {}
kmer_to_source = defaultdict(set)
for k in pep_lengths:
    curr_kmers = []
    for protein_id, seq in tqdm(seqs.items()):
        n = len(seq)
        if n < k:
            continue
    
        for i in range(n - k + 1):
            kmer = seq[i:i+k]
            curr_kmers.append(kmer)
            kmer_to_source[kmer].add(protein_id)
    kmers_by_size[k] = set(curr_kmers)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121766/121766 [02:36<00:00, 779.09it/s]


In [63]:
import numpy as np
aas = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
assert len(aas) == 20

def annotate_with_ref_epi_seq(df):
    df = df.copy()
    ref_peptides_col = []
    for _, row in df.iterrows():
        alt_peptide = row.ALT_EPI_SEQ

        pos = row.MUTATION_POSITION
        ref_peptides = set()
        if not np.isnan(pos):
            pos = int(pos)
            i = pos - 1
            alt_aa = alt_peptide[i]

            for aa in aas:
                if aa == alt_aa:
                    continue
                peptide = alt_peptide[:i] + aa + alt_peptide[i + 1:]
                if len(kmer_to_source[peptide]) > 0:
                    ref_peptides.add(peptide)
            if len(ref_peptides) == 0:
                for aa1 in aas:
                    for aa2 in aas:
                        peptide = alt_peptide[:i] + aa1 + aa2 + alt_peptide[i + 2:]

                        if peptide == alt_peptide:
                            continue
                        if len(kmer_to_source[peptide]) > 0:
                            print("MNV?", alt_peptide, aa1, aa2, aa3, peptide)
                            ref_peptides.add(peptide)
        ref_peptides = list(ref_peptides)
        ref_peptides_col.append(";".join(ref_peptides))
    df["ref_peptide_candidates"] = ref_peptides_col
    print("Unique: %d" % (len(df) - ((df.ref_peptide_candidates.str.len() == 0).sum() + df.ref_peptide_candidates.str.contains(";").sum()),))
    print ("Multiple: %d" % (df.ref_peptide_candidates.str.contains(";").sum(),))
    print("None: %d" % ((df.ref_peptide_candidates.str.len() == 0).sum(),))
    return df
    

In [64]:
df = annotate_with_ref_epi_seq(df)

MNV? RRHIEIRDK D Q Y RRHIEIRDQ
Unique: 558
Multiple: 16
None: 34


In [65]:
df.to_csv("tesla4_with_inferred_ref_peptides.csv")

In [66]:
df_7 = pd.read_excel("mmc7.xlsx")

In [67]:
df_7 = annotate_with_ref_epi_seq(df_7)

Unique: 299
Multiple: 9
None: 2


In [70]:
df_7.to_csv("tesla7_with_inferred_ref_peptides.csv")

In [71]:
df_7

,PMHC,PATIENT_ID,TISSUE_TYPE,ALT_EPI_SEQ,PEP_LEN,PREDICTED_BINDING_AFFINITY,NETMHC_BINDING_AFFINITY,TUMOR_ABUNDANCE,BINDING_STABILITY,AGRETOPICITY,FOREIGNNESS,MUTATION_POSITION,VALIDATED,TCR_FLOW_II,TCR_FLOW_II_QUANT,ref_peptide_candidates
0,A*01:01_AISDSLLWKY,8,TIL,AISDSLLWKY,10,62.900000,32.15,16.319886,1.65,0.164746,5.000000e-01,8,0,0,0.000,AISDSLLRKY
1,A*01:01_ASSSGTRLY,8,TIL,ASSSGTRLY,9,218.700000,334.79,3.024792,1.31,0.020218,0.000000e+00,9,0,0,0.000,ASSSGTRLH
2,A*01:01_ATDTNNLNVNY,9,TIL,ATDTNNLNVNY,11,67.400000,45.31,49.850000,4.19,0.350859,6.978296e-09,7,1,1,0.198,ATDTNNPNVNY
3,A*01:01_CSFRGSGSLSY,8,TIL,CSFRGSGSLSY,11,715.100000,648.89,0.392000,0.54,0.297983,2.049193e-13,9,0,0,0.000,CSFRGSGSPSY
4,A*01:01_CSTVKDFSY,8,TIL,CSTVKDFSY,9,186.700000,251.42,4.834086,0.76,0.017517,0.000000e+00,9,0,0,0.000,CSTVKDFSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,B*44:02_SELCPYNSV,9,TIL,SELCPYNSV,9,211.100000,180.66,8.212125,0.55,0.827843,5.000000e-01,8,0,0,0.000,SELCPYNPV
306,B*44:02_SELKATVEL,9,TIL,SELKATVEL,9,378.508142,286.83,30.409354,1.40,0.618112,0.000000e+00,5,0,0,0.000,SELKETVEL
307,B*44:02_SENNWAVGHKV,9,TIL,SENNWAVGHKV,11,526.700000,3063.66,8.207536,0.95,0.016995,9.999410e-01,2,0,0,0.000,SGNNWAVGHKV
308,B*44:02_VETDLQPFW,9,TIL,VETDLQPFW,9,70.900000,22.19,13.197900,1.38,0.003861,3.475804e-09,9,0,0,0.000,VETDLQPFR
